In [3]:
import pandas as pd
import numpy as np
from scipy.stats import chisquare
import os

# List of CSV files to process
base_path = r'C:\Users\willi\CAN_experiments\\'
csv_files = [
    'performance_metrics_correlated_signal.csv',
    'performance_metrics_max_speedometer.csv',
    'performance_metrics_reverse_light_off.csv',
    'performance_metrics_reverse_light_on.csv',
    'performance_metrics_max_engine_coolant.csv'
]

# Function to merge bins with zero or low expected frequencies
def merge_bins(observed_freq, expected_freq, threshold=5):
    """ Merge bins with expected frequencies below the threshold. """
    while any(expected_freq < threshold):
        for i in range(len(expected_freq)):
            if expected_freq[i] < threshold:
                if i == 0:
                    expected_freq[i + 1] += expected_freq[i]
                    observed_freq[i + 1] += observed_freq[i]
                elif i == len(expected_freq) - 1:
                    expected_freq[i - 1] += expected_freq[i]
                    observed_freq[i - 1] += observed_freq[i]
                else:
                    if expected_freq[i - 1] < expected_freq[i + 1]:
                        expected_freq[i - 1] += expected_freq[i]
                        observed_freq[i - 1] += observed_freq[i]
                    else:
                        expected_freq[i + 1] += expected_freq[i]
                        observed_freq[i + 1] += observed_freq[i]
                expected_freq = np.delete(expected_freq, i)
                observed_freq = np.delete(observed_freq, i)
                break
    return observed_freq, expected_freq

# Function to perform chi-squared test
def perform_chi_squared_test(df, version1, version2, bins=10, threshold=5):
    print("Function perform_chi_squared_test")
    
    # Extract the AUC-ROC values for both versions
    version1_values = df[df['version'] == version1]['roc_auc']
    version2_values = df[df['version'] == version2]['roc_auc']
    print("\nStep 1: Extract AUC-ROC values")
    print(f"Version 1 values ({version1}):\n", version1_values.head())
    print(f"Version 2 values ({version2}):\n", version2_values.head())
    
    # Discretize the continuous AUC-ROC values into bins
    #the goal here is to convert continuous AUC-ROC values into discrete bins for our comparison.
    observed_freq, bin_edges = np.histogram(version1_values, bins=bins) #so the np.histogram to bin the values.
    expected_freq, _ = np.histogram(version2_values, bins=bin_edges)
    print("\nStep 2: Discretize bins")
    print("Observed frequencies (version 1):", observed_freq)
    print("Bin edges:", bin_edges)
    print("Expected frequencies (version 2):", expected_freq)#to print our obs and exp freqs along with the bin edges
    
    # Adjust expected frequencies to ensure sums are equal
    #to ensurethat the sum of obs and exp freqs are equal.
    if observed_freq.sum() != expected_freq.sum():
        factor = observed_freq.sum() / expected_freq.sum()
        expected_freq = expected_freq * factor
        print("\nAdjusted Expected Frequencies:", expected_freq)
    
    # Print the version1 values and observed freqs
    print("\nVersion 1 Values (scaled):\n", version1_values.to_numpy() * 100, observed_freq)
    print("\nVersion 2 Values (scaled):\n", version2_values.to_numpy() * 100, expected_freq)

    # Merging bins where expected frequencies are too low to meet chi-squared test assumptions.
    observed_freq, expected_freq = merge_bins(observed_freq, expected_freq, threshold)
    print("\nStep 4: Merge bins with low expected frequencies")
    print("Merged observed frequencies:", observed_freq)
    print("Merged expected frequencies:", expected_freq)
    
    # Calculate degrees of freedom
    degrees_of_freedom = len(observed_freq) - 1
    print("Degrees of freedom:", degrees_of_freedom)
    
    # Perform the chi-squared test
    chi2_stat, p_val = chisquare(f_obs=observed_freq, f_exp=expected_freq)
    print("\nStep 5: Perform chi-squared test")
    print("Chi-squared statistic:", chi2_stat)
    print("P-value:", p_val)
    
    return chi2_stat, p_val, degrees_of_freedom

# Dictionary to store results
chi_squared_results = {}

# Perform chi-squared test for each attack type
for file in csv_files:
    file_path = os.path.join(base_path, file)
    df = pd.read_csv(file_path)
    attack_type = file.replace('performance_metrics_', '').replace('.csv', '')

    # Define the versions
    version1 = 'embeddings_only'
    version2 = 'normalized'

    # Perform chi-squared test
    chi2_stat, p_val, degrees_of_freedom = perform_chi_squared_test(df, version1, version2)

    # Store the results
    chi_squared_results[attack_type] = {
        'chi2_stat': chi2_stat,
        'p_val': p_val,
        'degrees_of_freedom': degrees_of_freedom
    }

# Print the results for each attack type
for attack_type, results in chi_squared_results.items():
    print(f"\nChi-squared test results for '{attack_type}' attack type:")
    print(f"Chi-squared statistic: {results['chi2_stat']}")
    print(f"P-value: {results['p_val']}")
    print(f"Degrees of freedom: {results['degrees_of_freedom']}")


Function perform_chi_squared_test

Step 1: Extract AUC-ROC values
Version 1 values (embeddings_only):
 0    0.915093
2    0.938126
4    0.927775
6    0.943781
8    0.961502
Name: roc_auc, dtype: float64
Version 2 values (normalized):
 1    0.962256
3    0.973305
5    0.950076
7    0.957467
9    0.986297
Name: roc_auc, dtype: float64

Step 2: Discretize bins
Observed frequencies (version 1): [14 13 10 17 11 13 10 11 10 10]
Bin edges: [0.89681196 0.90500789 0.91320382 0.92139975 0.92959567 0.9377916
 0.94598753 0.95418346 0.96237938 0.97057531 0.97877124]
Expected frequencies (version 2): [ 0  0  2  4  7 14 14 15 18 12]

Adjusted Expected Frequencies: [ 0.          0.          2.76744186  5.53488372  9.68604651 19.37209302
 19.37209302 20.75581395 24.90697674 16.60465116]

Version 1 Values (scaled):
 [91.5092893  93.81264857 92.77750454 94.37811665 96.15015498 92.49423259
 94.81742814 90.44147051 95.8361179  93.67485892 91.68313688 93.41043317
 92.18250463 96.21087018 92.24254232 94.0672